In [10]:
import cv2
import numpy as np

def compute_snr(image_path, noise_roi, signal_roi):
    """
    Compute the SNR of an image given specific ROIs for noise and signal.

    Parameters:
    - image_path: path to the image file.
    - noise_roi: a tuple (x, y, width, height) defining the ROI for noise.
    - signal_roi: a tuple (x, y, width, height) defining the ROI for signal.

    Returns:
    - SNR in dB.
    """

    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Extract ROIs
    noise_region = img[noise_roi[1]:noise_roi[1]+noise_roi[3], noise_roi[0]:noise_roi[0]+noise_roi[2]]
    signal_region = img[signal_roi[1]:signal_roi[1]+signal_roi[3], signal_roi[0]:signal_roi[0]+signal_roi[2]]

    # Compute mean and standard deviation for noise
    mu_noise = np.mean(noise_region)
    sigma_noise = np.std(noise_region)

    # Compute mean for signal
    mu_signal = np.mean(signal_region)

    # Compute SNR in dB
    snr_db = 10 * np.log10(mu_signal**2 / sigma_noise**2)

    return snr_db

# Example usage:
image_path_good = r"C:\Users\montruth\fishPy\tests\f2\results\anatomy\plane03\anatomy_binned_x1y1z2_20220511_RM0008_126hpf_fP10_f2_t1_o1Ala_001_.tif"

image_path_low = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-20\f3\results\anatomy\plane03\anatomy_binned_x1y1z1_20220420_RM0012_124hpf_fP8_f3_t1_o1Ala_001_.tif"
noise_roi = (230, 260, 10, 10)  # Define these coordinates appropriately
signal_roi = (70, 190, 10, 10)  # Define these coordinates appropriately
print(compute_snr(image_path, noise_roi, signal_roi))

13.038087239948212


<Image layer 'img' at 0x17f6a885b20>

In [23]:
import cv2
import numpy as np
from numpy.fft import fftshift, fft2

def compute_gradient_magnitude(image_path):
    """ The magnitude of the gradient can be used to estimate image sharpness. Sharp images tend to have higher gradient magnitudes, especially at edges."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    return np.mean(magnitude)

def compute_laplacian_variance(image_path):
    """  The variance of the Laplacian of an image has been used as a sharpness metric. The idea is that sharper images will have a wider range of second derivative values."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    return np.var(laplacian)

def frequency_content(image_path):
    """In the frequency domain (after a Fourier transform), the amount of high-frequency content can be related to image sharpness"""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    f_transform = fftshift(fft2(image))
    magnitude_spectrum = np.abs(f_transform)
    # For simplicity, consider the sum of high-frequency components
    center_x, center_y = magnitude_spectrum.shape[0] // 2, magnitude_spectrum.shape[1] // 2
    return np.sum(magnitude_spectrum[center_x-10:center_x+10, center_y-10:center_y+10])

def tenengrad_measure(image_path, threshold=100):
    """It is based on the sum of squared gradients. The threshold to exclude minor gradients."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    return np.sum(magnitude > threshold)



def brenner_gradient(image_path):
    """It measures sharpness based on the squared difference in intensity between pixels separated by a certain distance."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    shifted_image = np.roll(image, shift=-2, axis=0)
    brenner = np.sum((shifted_image - image)**2)
    return brenner

def energy_of_gradient(image_path):
    """This metric simply involves calculating the gradient magnitude for each pixel in the image and then summing the squared values. A sharp image would have a higher EoG value since sharper images have more pronounced edges and higher gradient magnitudes"""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = gradient_x**2 + gradient_y**2
    return np.sum(magnitude)

def cumulative_probability_of_blur_detection(image_path):
    """CPBD is a more complex measure which evaluates sharpness based on a combination of image gradients and a perceptual thresholding function. In a sharp image, more gradients will exceed the perceptual threshold, leading to a higher CPBD value.It's a multi-step process, and here's a simplified version:"""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(gradient_x**2 + gradient_y**2)

    # Define perceptual threshold function (this can be adjusted)
    threshold = np.mean(magnitude) + np.std(magnitude)

    # Count gradients above threshold
    count = np.sum(magnitude > threshold)

    # Compute probability (ratio of counted gradients to total pixels)
    probability = count / (image.shape[0] * image.shape[1])

    return probability



In [28]:

image_path_good = r"C:\Users\montruth\fishPy\tests\f2\results\anatomy\plane02\anatomy_binned_x1y1z2_20220511_RM0008_126hpf_fP10_f2_t1_o1Ala_001_.tif"
image_path_medium = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-02-14\f3\results\anatomy\plane02\anatomy_binned_x1y1z1_20220214_RM0012_fP14_130hpf_f3_t1_o1Ala_001_.tif"
image_path_low = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-20\f3\results\anatomy\plane02\anatomy_binned_x1y1z1_20220420_RM0012_124hpf_fP8_f3_t1_o1Ala_001_.tif"
path_list =[image_path_good, image_path_medium, image_path_low]
quali_list = ["good", "medium", "low"]
for i, path_i in enumerate(path_list):
    print(quali_list[i])
    print("gradient_magnitude",compute_gradient_magnitude(path_i))
    print("laplacian_variance",compute_laplacian_variance(path_i))
    print("frequency_content",frequency_content(path_i))
    print("tenengrad_measure",tenengrad_measure(path_i,100))
    print("brenner_gradient",brenner_gradient(path_i))
    print("energy_of_gradient",energy_of_gradient(path_i))
    print("cumulative_probability_of_blur_detection",cumulative_probability_of_blur_detection(path_i))
    print("SNR", compute_snr(path_i, noise_roi, signal_roi))
    print("----------")

good
gradient_magnitude 26.93131100359268
laplacian_variance 442.64243278487993
frequency_content 53015234.8808965
tenengrad_measure 4338
brenner_gradient 10653040
energy_of_gradient 349897506.0
cumulative_probability_of_blur_detection 0.11472320556640625
SNR -1.759198862397606
----------
medium
gradient_magnitude 15.418368028622886
laplacian_variance 127.6871922572609
frequency_content 41733478.9864351
tenengrad_measure 2903
brenner_gradient 4514226
energy_of_gradient 184188432.0
cumulative_probability_of_blur_detection 0.08612823486328125
SNR -14.46040579733928
----------
low
gradient_magnitude 19.769377985129772
laplacian_variance 265.1668178474065
frequency_content 57871042.29488559
tenengrad_measure 2409
brenner_gradient 7384124
energy_of_gradient 209570040.0
cumulative_probability_of_blur_detection 0.10692977905273438
SNR 7.0763265494690994
----------


In [27]:
import napari
viewer = napari.Viewer()
img = cv2.imread(image_path_good, cv2.IMREAD_GRAYSCALE)
viewer.add_image(img, name= "good")
img = cv2.imread(image_path_medium, cv2.IMREAD_GRAYSCALE)
viewer.add_image(img, name = "medium")
img = cv2.imread(image_path_low, cv2.IMREAD_GRAYSCALE)
viewer.add_image(img, name = "low")

<Image layer 'low' at 0x17f0a109e50>